## Modelos de ML para predecir la probabilidad de que una persona fallezca por covid 19 dados ciertos factores

Este notebook está inspirado en el ejemplo de Ken Jee sobre la base de datos del titanic https://www.kaggle.com/kenjee/titanic-project-example.
Aquí, sigo, casi al pie de la letra, su procedimiento a modo de aprendizaje y práctica.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
raw_data = pd.read_csv("210316COVID19MEXICO.csv", parse_dates=True)

Únicamente me voy a quedar con datos de la alcaldía Gustavo a Madero en la CDMX para no trabajar con una base de datos demasiado grande. De la misma manera, solo me quedo con resultados positivos por pruebas pcr de laboratorio para evitar problemas con la selección de los datos

In [3]:
raw_data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'INDIGENA',
       'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'TOMA_MUESTRA_LAB', 'RESULTADO_LAB',
       'TOMA_MUESTRA_ANTIGENO', 'RESULTADO_ANTIGENO', 'CLASIFICACION_FINAL',
       'MIGRANTE', 'PAIS_NACIONALIDAD', 'PAIS_ORIGEN', 'UCI'],
      dtype='object')

In [4]:
raw_data.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2021-03-16,z482b8,1,12,9,2,9,9,12,1,...,2,2,97,2,97,1,99,México,97,97
1,2021-03-16,z49a69,1,12,23,1,23,23,4,2,...,1,2,97,2,97,2,99,México,97,1
2,2021-03-16,z23d9d,1,12,22,2,24,22,9,1,...,2,2,97,2,97,6,99,México,97,97
3,2021-03-16,z24953,1,12,9,1,9,9,10,1,...,1,1,2,2,97,7,99,México,97,97
4,2021-03-16,zz8e77,1,12,9,2,9,9,2,1,...,2,2,97,2,97,6,99,México,97,97


In [21]:
df = raw_data[raw_data['ENTIDAD_RES']==9]

In [22]:
df = df[df['MUNICIPIO_RES']==5]

In [23]:
df = df[df['RESULTADO_LAB']==1]

In [13]:
df.describe()

,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,INTUBADO,NEUMONIA,...,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,UCI
count,34519.000000,34519.000000,34519.000000,34519.000000,34519.000000,34519.0,34519.0,34519.000000,34519.000000,34519.000000,...,34519.000000,34519.000000,34519.000000,34519.0,34519.0,34519.000000,34519.000000,34519.0,34519.000000,34519.000000
mean,1.750659,9.277847,9.172369,1.503983,10.762334,9.0,5.0,1.201483,77.823402,1.834931,...,2.088560,2.002549,7.458096,1.0,1.0,1.891219,86.645992,3.0,98.774501,77.848344
std,0.432638,3.700982,1.125921,0.499991,6.053971,0.0,0.0,0.401114,38.177263,0.371248,...,3.144069,3.243233,23.380644,0.0,0.0,0.311368,29.637067,0.0,4.678816,38.127356
min,1.000000,1.000000,1.000000,1.000000,1.000000,9.0,5.0,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.000000,3.0,1.000000,1.000000
25%,2.000000,4.000000,9.000000,1.000000,9.000000,9.0,5.0,1.000000,97.000000,2.000000,...,2.000000,2.000000,1.000000,1.0,1.0,2.000000,97.000000,3.0,99.000000,97.000000
50%,2.000000,12.000000,9.000000,2.000000,9.000000,9.0,5.0,1.000000,97.000000,2.000000,...,2.000000,2.000000,2.000000,1.0,1.0,2.000000,97.000000,3.0,99.000000,97.000000
75%,2.000000,12.000000,9.000000,2.000000,9.000000,9.0,5.0,1.000000,97.000000,2.000000,...,2.000000,2.000000,2.000000,1.0,1.0,2.000000,97.000000,3.0,99.000000,97.000000
max,2.000000,12.000000,32.000000,2.000000,99.000000,9.0,5.0,2.000000,97.000000,2.000000,...,98.000000,98.000000,99.000000,1.0,1.0,2.000000,97.000000,3.0,99.000000,97.000000


In [10]:
df.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'INDIGENA',
       'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'TOMA_MUESTRA_LAB', 'RESULTADO_LAB',
       'TOMA_MUESTRA_ANTIGENO', 'RESULTADO_ANTIGENO', 'CLASIFICACION_FINAL',
       'MIGRANTE', 'PAIS_NACIONALIDAD', 'PAIS_ORIGEN', 'UCI'],
      dtype='object')

In [24]:
df.FECHA_DEF.value_counts()

9999-99-99    31037
2020-06-12       27
2021-01-02       24
2020-05-31       23
2020-06-17       23
              ...  
2020-04-17        1
2020-04-02        1
2020-10-07        1
2020-04-14        1
2020-03-29        1
Name: FECHA_DEF, Length: 346, dtype: int64

In [26]:
def defuncion(x):
    '''Función que indica si el paciente falleció o no'''
    if x == '9999-99-99':
        d = 0
    else:
        d=1
    return d

In [29]:
#Agremaos una columna al dataframe que indique si el paciente falleció
df['DEF'] = df['FECHA_DEF'].apply(defuncion)

In [28]:
features = ['SEXO', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'EMBARAZO','DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO', 'DEF']

In [30]:
df_gam = df[features]

In [31]:
df_gam.head()

,SEXO,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,INTUBADO,NEUMONIA,EDAD,EMBARAZO,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,OTRA_COM,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,DEF
55,1,1,2020-04-22,2020-04-20,9999-99-99,97,2,45,2,2,2,2,2,2,2,2,1,2,2,0
24599,1,2,2020-03-12,2020-03-11,9999-99-99,2,2,25,98,2,2,2,2,2,2,2,2,2,2,0
29477,2,2,2020-03-24,2020-03-24,9999-99-99,2,1,48,97,2,2,2,2,2,2,2,2,2,2,0
29643,1,2,2020-03-20,2020-03-20,2020-04-11,2,1,51,2,2,2,2,2,2,2,2,2,2,2,1
29911,2,2,2020-03-23,2020-03-23,2020-04-10,1,2,70,97,2,2,2,2,2,2,2,2,2,2,1


In [18]:
df_gam.describe()

,SEXO,TIPO_PACIENTE,INTUBADO,NEUMONIA,EDAD,EMBARAZO,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,OTRA_COM,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO
count,34519.000000,34519.000000,34519.000000,34519.000000,34519.000000,34519.000000,34519.000000,34519.000000,34519.000000,34519.000000,34519.000000,34519.000000,34519.000000,34519.000000,34519.000000,34519.000000
mean,1.503983,1.201483,77.823402,1.834931,44.908862,50.188012,1.987340,2.088531,2.082679,2.079290,1.941684,2.085866,2.076016,1.929343,2.088560,2.002549
std,0.499991,0.401114,38.177263,0.371248,17.246986,47.503726,3.407742,3.100876,3.145164,2.923548,3.334004,3.187178,3.015039,3.083455,3.144069,3.243233
min,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,97.000000,2.000000,32.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
50%,2.000000,1.000000,97.000000,2.000000,44.000000,97.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
75%,2.000000,1.000000,97.000000,2.000000,57.000000,97.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
max,2.000000,2.000000,97.000000,2.000000,109.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000


2    28821
1     5698
Name: NEUMONIA, dtype: int64